In [1]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import requests
import datetime

In [20]:
binance_api_key = 'E90xR7UnO2oeP3lrgC50zkqUuAqyyXs4uSlAGcqIVO9n90ELrFI4gfVI2HGGxxCj'
binance_api_secret='Inls2xYl9FqPi0zlIrucfxG81drx7w6Pv6SzYFxMNoAcwDjlCKug7zUM9zB3lF9h'

In [21]:
client = Client(binance_api_key,binance_api_secret)

In [22]:
def get_market_cap(quote="USDT"):
    
    resp = requests.get("https://www.binance.com/bapi/asset/v2/public/asset-service/product/get-products")
    
    market_cap=pd.DataFrame(resp.json()['data'])
    market_cap=market_cap[market_cap['q']==quote]
    
    market_cap=market_cap[['an','qn','s','b','q','c','cs']]
    market_cap['c']=market_cap['c'].astype(float)
    
    market_cap.columns=['Long name','Quote Name','Ticker','Short Name','Quote Short Name','Close','Supply']
    market_cap['Market Cap']=market_cap['Close']*market_cap['Supply']
    
    market_cap=market_cap.sort_values(by='Market Cap',ascending=False)
    

    return market_cap

In [23]:

def get_price(ticker_list,date=str(datetime.date.today())):
    

    price=pd.DataFrame()

    for ticker in ticker_list:

        temp=pd.DataFrame()
        
        numeric_columns =  ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                        'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']
        
        data = pd.DataFrame(client.get_historical_klines(ticker, Client.KLINE_INTERVAL_1DAY, date),columns=numeric_columns)
        data['Close Time']=pd.to_datetime(data['Close Time'], unit='ms').dt.date
        data=data.set_index('Close Time')

        temp[ticker]=data['Close'].astype(float)
        
        #price=pd.concat([price,temp[ticker]],axis=1)
        price=pd.concat([price,temp[ticker]],axis=1)
        
    return price

In [24]:
def get_inventory():

    ticker_info = client.get_all_tickers()

    ticker_prices = {ticker['symbol']: float(ticker['price']) for ticker in ticker_info}
    eurusdt=ticker_prices.get("EURUSDT")
    ticker_prices=pd.DataFrame(ticker_prices.values(),index=ticker_prices.keys(),columns=['Price in €'])


    ptf=pd.DataFrame(client.get_user_asset())
    ptf['Ticker']=ptf['asset']+"USDT"

    ticker=ptf['Ticker'].to_list()
    ptf=ptf.set_index('Ticker')

    inventory=ticker_prices.loc[ticker]

    inventory['Quantity']=ptf['free'].astype(float)+ptf['locked'].astype(float)
    inventory['Position']=inventory['Quantity']*inventory['Price in €']
    inventory['Weights in %']=inventory['Position']/inventory['Position'].sum()*100

    inventory.loc['Total']=['','',inventory['Position'].sum(),inventory['Weights in %'].sum()]

    return inventory.sort_values(by='Weights in %',ascending=False)
    
    

In [39]:
plo=client.get_account_snapshot(type='SPOT')

In [54]:
plo['snapshotVos'][2]['data']

{'totalAssetOfBtc': '0.00126825',
 'balances': [{'asset': 'BTC', 'free': '0.00040885', 'locked': '0'},
  {'asset': 'ETH', 'free': '0.00840723', 'locked': '0'},
  {'asset': 'SOL', 'free': '0.18668057', 'locked': '0'}]}